# GO FROM ONE WAYPOINT TO THE NEXT AND STOP
María Rodríguez Palomo

In [1]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### CREATE WORLD

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### VIEW CHANGE

In [3]:
loc2=carla.Transform(carla.Location(x=-174.557922, y=185.525208, z=10.138501), carla.Rotation(pitch=0.733207, yaw=-147.194565, roll=0.006472))
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc2.location + carla.Location(x=10,y=5,z=4),carla.Rotation(yaw=loc2.rotation.yaw))
spectator.set_transform(spectator_pos)

#### CREATE VEHICLE

In [4]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### CONSTANT SPEED
A function to maintain a constant speed

In [5]:
def maintain_speed(s,preferred_speed,speed_threshold):
    #The return is the acceleration
    if s>=10:
        return 0
    elif s < 10 - 2:
        return 0.8
    else:
        return 0.4

#### ANGLE
A function that calculates the angle between two vectors

In [6]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    print(car_x)
    print(car_y)
    print(wp_x)
    print(wp_y)
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [7]:
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### INITIAL AND NEXT LOCATION
Display the current location on the screen and the next point a few meters ahead on the same lane

In [12]:
print("Initial location: ", vehicle.get_transform())
loc=vehicle.get_location()
current_w = map.get_waypoint(loc)
#Calculation of the next point to go to
waypoint_separation = 5
next_w0 = list(current_w.next(waypoint_separation))[0]
next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
print("next: ",next_w.transform)

Initial location:  Transform(Location(x=-188.032059, y=178.176758, z=9.125488), Rotation(pitch=0.642297, yaw=-155.457611, roll=-0.078400))
next:  Transform(Location(x=-191.425537, y=175.879074, z=9.171032), Rotation(pitch=360.604553, yaw=214.141022, roll=0.000000))


#### IMPLEMENT ANGLE AND SPEED TO THE CAR

In [13]:
preferred_speed = 90
speed_threshold = 2
MAX_STEER_DEGREES = 5
velocity = vehicle.get_velocity()
speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
predicted_angle = get_angle(vehicle,next_w)
steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
#Aplicamos los cálculos al coche
vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))
while True:
    if vehicle.get_transform().location.x<= next_w.transform.location.x and vehicle.get_transform().location.y<= next_w.transform.location.y:
        vehicle.apply_control(carla.VehicleControl(brake=1.0,throttle= 0, steer=0))
        print("We are in next")
        break

-188.0320587158203
178.1767578125
-191.425537109375
175.8790740966797
Hemos llegado a next


#### DESTROY

In [14]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()